In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F


# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(), download=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified
Files already downloaded and verified


In [34]:

# # Define deepfool attack function
# def deepfool_attack(image, model, num_classes=10, overshoot=0.02, max_iter=50):
#     """
#     Deepfool attack implementation for CIFAR-10 images.
#     """
#     # Convert image to tensor and send to device
#     image = image.to(device)
#     image.requires_grad = True
    
#     # Get predicted label for image
#     output = model(image)
#     init_pred = output.max(1, keepdim=True)[1].item()
    
#     # Initialize perturbation
#     perturbation = torch.zeros_like(image).to(device)
#     perturbed_image = torch.zeros_like(image).to(device)
    
#     # Initialize counter and loop until the image is misclassified
#     i = 0
#     while init_pred == output.max(1, keepdim=True)[1].item() and i < max_iter:
#         # Calculate gradients and get predicted label
#         output.backward()
#         grad = image.grad.data
#         pred_label = output.max(1, keepdim=True)[1].item()
        
#         # Initialize min and min_index
#         min_norm = float('inf')
#         min_index = None
        
#         # Loop through all classes
#         for k in range(num_classes):
#             # Skip if class is equal to predicted class
#             if k == pred_label:
#                 continue
            
#             # Calculate perturbation for class k
#             w_k = grad[k, :, :, :].clone()
#             w = grad[pred_label, :, :, :].clone()
#             f_k = output[0, k].item()
#             f = output[0, pred_label].item()
#             pert_k = abs(f_k - f) / torch.norm(w_k - w)
            
#             # Update min_norm and min_index if perturbation for class k is smaller
#             if pert_k < min_norm:
#                 min_norm = pert_k
#                 min_index = k
        
#         # Update perturbation and perturbed_image
#         r = min_norm * (w - w_k)
#         perturbation += r
#         perturbed_image = image + (1 + overshoot) * perturbation
        
#         # Clamp the perturbed image to valid range
#         perturbed_image = torch.clamp(perturbed_image, 0, 1).detach()
        
#         # Reset gradients and increment counter
#         model.zero_grad()
#         output = model(perturbed_image)
#         i += 1
        
#     return perturbed_image.detach(), init_pred, output.max(1, keepdim=True)[1].item()



def deepfool_attack(image, model, num_classes=10, overshoot=0.02, max_iter=50):
    """
    Deepfool attack implementation for CIFAR-10 images.
    """
    # Convert image to tensor and send to device
    # image = image.unsqueeze(0).to(device)
    image = image.to(device)
    image.requires_grad = True
    
    # Get predicted label for image
    with torch.enable_grad():
        output = model(image)
    # init_pred = output.max(1, keepdim=True)[1].item()
    
    # Check if output tensor is empty
    if output.numel() == 0:
        return None, None, None
    output_sum = output.sum()
    init_pred = output.max(1, keepdim=True)[1].item()
    # Initialize perturbation
    perturbation = torch.zeros_like(image).to(device)
    perturbed_image = torch.zeros_like(image).to(device)
    
    # Initialize counter and loop until the image is misclassified
    i = 0
    while init_pred == output.max(1, keepdim=True)[1].item() and i < max_iter and output.numel() > 0:

        # Calculate gradients and get predicted label
        image.grad = torch.zeros_like(image)
        grad = image.grad.data
        
        output.backward(torch.ones_like(output), retain_graph=True)
        grad = image.grad.clone().detach()

        pred_label = output.max(1, keepdim=True)[1].item()
        
        # Initialize min and min_index
        min_norm = float('inf')
        min_index = None
        
        # Loop through all classes
        for k in range(num_classes):
            # Skip if class is equal to predicted class
            if k == pred_label:
                continue
            
            # Calculate perturbation for class k
            w_k = grad.clone()
            w_k[0, pred_label] = 0
            w = grad.clone()
            w[0, k] = 0


            f_k = output[0, k].item()
            f = output[0, pred_label].item()
            pert_k = abs(f_k - f) / torch.norm(w_k - w)
            
            # Update min_norm and min_index if perturbation for class k is smaller
            if pert_k < min_norm:
                min_norm = pert_k
                min_index = k
        
        # Update perturbation and perturbed_image
        r = min_norm * (w - w_k)
        perturbation += r
        perturbed_image = image + (1 + overshoot) * perturbation

        # Clamp the perturbed image to valid range
        perturbed_image = torch.clamp(perturbed_image, 0, 1).detach()


        
        # # Update perturbation and perturbed_image
        # r = min_norm * (w - w_k)
        # perturbation += r
        # perturbed_image = image + (1 + overshoot) * perturbation
        
        # # Clamp the perturbed image to valid range
       


        
        # # Update perturbation and perturbed_image
        # r = min_norm * (w - w_k)
        # perturbation += r
        # perturbed_image = image + (1 + overshoot) * perturbation
        
        # # Clamp the perturbed image to valid range
        # perturbed_image = torch.clamp(perturbed_image, 0, 1).detach()
        
        # Reset gradients and increment counter
        model.zero_grad()
        with torch.enable_grad():
            output = model(perturbed_image)
            output_sum = output.sum()
        i += 1



        
    return perturbed_image.detach(), init_pred, output.max(1, keepdim=True)[1].item()




In [35]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * 128, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(-1, 4 * 4 * 128)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torchvision.models as models

# class DenseNet(nn.Module):
#     def __init__(self):
#         super(DenseNet, self).__init__()
#         self.densenet = models.densenet121(pretrained=True)
#         self.fc = nn.Linear(1000, 10)

#     def forward(self, x):
#         x = self.densenet(x)
#         x = self.fc(x)
#         return x

model = CNN().to(device)



# Initialize model and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)



In [36]:
log_interval=10
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if data.shape[0] == 1:
            continue
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    return model





In [37]:
# def test(model, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     accuracy = 100. * correct / len(test_loader.dataset)
#     print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)")
#     return accuracy


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Apply deepfool attack on first image in batch
            perturbed_image, init_pred, adv_pred = deepfool_attack(data[0], model)

            # Get predicted label for perturbed image
            output = model(perturbed_image.unsqueeze(0))
            adv_pred = output.max(1, keepdim=True)[1].item()

            # Calculate loss and accuracy
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%), Adv Accuracy: {len(test_loader.dataset) - correct}/{len(test_loader.dataset)} '
          f'({100 - accuracy:.2f}%)')
    return accuracy


In [38]:

# Initialize Federated Learning setup
num_clients = 10
train_loaders = []
test_loaders = []



trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_subsplits = torch.utils.data.random_split(trainset, [int(len(train_dataset)/num_clients) for _ in range(num_clients)])
test_subsplits = torch.utils.data.random_split(testset, [int(len(test_dataset)/num_clients) for _ in range(num_clients)])

for i in range(num_clients):
    train_loader = torch.utils.data.DataLoader(train_subsplits[i], batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_subsplits[i], batch_size=128, shuffle=False)
    
    train_loaders.append(train_loader)
    test_loaders.append(test_loader)



# Define Federated Learning function
def federated_learning(model, train_loaders, test_loaders, optimizer, num_epochs):
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}\n-------------------------------")
        for i in range(num_clients):
            print(f"Client {i + 1}")
            model = train(model, train_loaders[i], optimizer, epoch + 1)
        test_accs = []
        for i in range(num_clients):
            test_acc = test(model, test_loaders[i])
            test_accs.append(test_acc)
        print(f"Test Accuracy: {sum(test_accs) / len(test_accs)}")
    
    return model

# Train the model using Federated Learning
model = federated_learning(model, train_loaders, test_loaders, optimizer, num_epochs=100)


Streaming output truncated to the last 5000 lines.
Train Epoch: 21 [2560/5000 (50%)]	Loss: 2.302475
Train Epoch: 21 [3840/5000 (75%)]	Loss: 2.302791
Client 9
Train Epoch: 21 [0/5000 (0%)]	Loss: 2.302426
Train Epoch: 21 [1280/5000 (25%)]	Loss: 2.302612
Train Epoch: 21 [2560/5000 (50%)]	Loss: 2.302878
Train Epoch: 21 [3840/5000 (75%)]	Loss: 2.302536
Client 10
Train Epoch: 21 [0/5000 (0%)]	Loss: 2.302413
Train Epoch: 21 [1280/5000 (25%)]	Loss: 2.302612
Train Epoch: 21 [2560/5000 (50%)]	Loss: 2.302891
Train Epoch: 21 [3840/5000 (75%)]	Loss: 2.302477
Test set: Average loss: 2.3026, Accuracy: 121/1000 (12.10%), Adv Accuracy: 879/1000 (87.90%)
Test set: Average loss: 2.3026, Accuracy: 102/1000 (10.20%), Adv Accuracy: 898/1000 (89.80%)
Test set: Average loss: 2.3026, Accuracy: 117/1000 (11.70%), Adv Accuracy: 883/1000 (88.30%)
Test set: Average loss: 2.3025, Accuracy: 119/1000 (11.90%), Adv Accuracy: 881/1000 (88.10%)
Test set: Average loss: 2.3025, Accuracy: 128/1000 (12.80%), Adv Accuracy: 8